In [1]:
import sys, os
import pandas as pd
import pathSetting
sys.path.append("../../..")

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import meta as ml_meta

import torch

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


In [2]:
# Set regression mode i.e., 'regression','forecast' 
mode_selection = "regression"

# Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
model_method = 'FC_rg'

# 2. Training 

## 2-1. Data selection

In [3]:
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [4]:
dataList

['IntegraionTrainX',
 'IntegraionTrainy',
 'IntegraionTestX',
 'IntegraionTesty',
 'trainClean_Hs1SwineFarmWithWeatherTime',
 'trainNoClean_Hs1SwineFarmWithWeatherTime',
 'testClean_Hs1SwineFarmWithWeatherTime',
 'testNoClean_Hs1SwineFarmWithWeatherTime']

In [5]:
# for regression data
if mode_selection == 'regression':
    #2
    # dataX
    dataName_X = dataList[0]
    # dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

    # datay
    dataName_y = dataList[1]
    # dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

    #3
    dataFolderName = "data_integrated_result"
    current = os.getcwd()
    dataFolderPath = os.path.join(current, dataFolderName)
    dataX = ml_data.get_saved_integrated_data('CSV', dataName_X, dataFolderPath)
    datay = ml_data.get_saved_integrated_data('CSV', dataName_y, dataFolderPath)
    integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

# for forecast data test
elif mode_selection == 'forecast':
    cleanParamList = ['Clean', 'NoClean']
    cleanMode = cleanParamList[1]

    datasetNameList = ['Hs1SwineFarmWithWeatherTime', 'gunwiStrawberryWithWeatherTime', 'strawberryOpenTime']
    datasetName = datasetNameList[0]

    dataName_X = 'train' + cleanMode + '_' + datasetName
    dataSaveMode_X = DataMeta[dataName_X]['integrationInfo']['DataSaveMode']

    dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, pathSetting.dataFolderPath)
    integration_freq_sec = DataMeta[dataName_X]['integrationInfo']['integration_freq_sec']

In [6]:
print(type(dataX), dataX.shape)
print(dataX)

<class 'pandas.core.frame.DataFrame'> (13680, 24)
                                RH_1       RH_2       RH_3       RH_4  \
datetime                                                                
2016-01-11 00:00:00+00:00  31.666667  31.566667  34.230000  31.000000   
2016-01-11 00:10:00+00:00  31.926667  31.700000  34.290000  31.066667   
2016-01-11 00:20:00+00:00  32.060000  31.956667  34.400000  31.230000   
2016-01-11 00:30:00+00:00  32.260000  32.223333  34.400000  31.356667   
2016-01-11 00:40:00+00:00  32.333333  32.450000  34.400000  31.500000   
...                              ...        ...        ...        ...   
2016-04-14 23:10:00+00:00  32.593333  32.290000  35.000000  30.796667   
2016-04-14 23:20:00+00:00  32.066667  31.926667  34.400000  30.930000   
2016-04-14 23:30:00+00:00  31.760000  31.600000  34.126667  30.723333   
2016-04-14 23:40:00+00:00  31.426667  31.463333  34.090000  30.890000   
2016-04-14 23:50:00+00:00  31.533333  31.390000  34.163333  30.890000   



In [7]:
# print(type(datay), datay.shape)
# print(datay)

## 2-2. Training Data Preparation

In [8]:
# for regression
if mode_selection == 'regression':
    # 2 Training Data Preparation
    # 2-1
    featureListX= list(dataX.columns)
    featureListy= list(datay.columns)

    # 2-2
    cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

    # 2-2-1 cleanTrainDataParam == Clean 일 경우
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }
    # 2-3
    scalerParam='scale'
    scaleMethod='minmax'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
    scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam, scaleMethod)

# for forecast
elif mode_selection == 'forecast':
    # 2-1
    featureListX = ['Temperature', 'out_temp','sin_hour']
    featureListy = 'Temperature'

    # 2-2
    cleanTrainDataParam = cleanMode

    # 2-2-1 cleanMode == Clean 일 때만 활성화
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }

    # 2-3
    scalerParam='scale'
    scaleMethod ='robust'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanMode)
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_x, val_x = ml_clean.delete_low_quality_train_val_data(train_x, val_x, cleanMode, NaNProcessingParam)

['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Press_mm_hg', 'RH_out', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File


In [9]:
print(type(train_x), train_x.shape)
print(train_x)

<class 'pandas.core.frame.DataFrame'> (10944, 24)
                               RH_1      RH_2      RH_3      RH_4      RH_5  \
datetime                                                                      
2016-01-11 00:00:00+00:00  0.127786  0.324722  0.255336  0.102083  0.230779   
2016-01-11 00:10:00+00:00  0.134942  0.328622  0.258140  0.105062  0.228774   
2016-01-11 00:20:00+00:00  0.138611  0.336128  0.263281  0.112363  0.229225   
2016-01-11 00:30:00+00:00  0.144115  0.343927  0.263281  0.118024  0.228323   
2016-01-11 00:40:00+00:00  0.146133  0.350556  0.263281  0.124430  0.226819   
...                             ...       ...       ...       ...       ...   
2016-03-26 23:10:00+00:00  0.201174  0.328037  0.253934  0.148118  0.121567   
2016-03-26 23:20:00+00:00  0.188148  0.320199  0.253934  0.138881  0.116991   
2016-03-26 23:30:00+00:00  0.185396  0.330209  0.253934  0.147820  0.111853   
2016-03-26 23:40:00+00:00  0.185396  0.338448  0.256337  0.167486  0.109323   
20

In [10]:
print(type(val_x), val_x.shape)
print(val_x)
# print(type(train_y), train_y.shape)
# print(train_y)

<class 'pandas.core.frame.DataFrame'> (2736, 24)
                               RH_1      RH_2      RH_3      RH_4      RH_5  \
datetime                                                                      
2016-03-27 00:00:00+00:00  0.267957  0.504094  0.338059  0.367272  0.216294   
2016-03-27 00:10:00+00:00  0.266306  0.502486  0.334943  0.372933  0.216294   
2016-03-27 00:20:00+00:00  0.266306  0.500877  0.333385  0.375913  0.216294   
2016-03-27 00:30:00+00:00  0.267957  0.500877  0.336501  0.379190  0.216745   
2016-03-27 00:40:00+00:00  0.277039  0.507799  0.339461  0.389321  0.217647   
...                             ...       ...       ...       ...       ...   
2016-04-14 23:10:00+00:00  0.153289  0.345876  0.291323  0.092995  0.245539   
2016-04-14 23:20:00+00:00  0.138795  0.335251  0.263281  0.098954  0.241304   
2016-04-14 23:30:00+00:00  0.130355  0.325697  0.250506  0.089717  0.239299   
2016-04-14 23:40:00+00:00  0.121182  0.321700  0.248793  0.097166  0.235841   
201

In [11]:
def transDFtoNP_yk(dfX, dfy, windowNum = 0, dim = None):
    """
    Make NumpyArray by input DataFrame.
    if windowNum = 0 ----> slice X by day
    if windowNum = N ----> slice X by windowNum
    
    Example:
        >>> Retunrn 
        ... X.shape (sampleNum, featureNum, sequenceNum )
        ... y.shape (sampleNum, )

    Args:
        dfX (DataFrame): dfX
        dfy (DataFrame): dfy
        windowNum (Interger): windowNum

    Returns:
        numpy array:  X, y
    
    """
    import datetime as dt
    import numpy as np

    if dim == 2:
        X = dfX.to_numpy()
        y = np.array(dfy.squeeze().tolist())
    else:
        X =[]
        y= []

        if windowNum ==0:
            dateList = dfX.index.map(lambda t: t.date()).unique()
            for startDate in dateList:
                endDate  = dt.datetime.combine(startDate, dt.time(23, 59, 59, 59))
                dfX_partial = dfX[startDate:endDate]
                dfy_partial = dfy[startDate:endDate]
                X_partial = dfX_partial.values
                y_partial = dfy_partial.values[0][0]
                X.append (X_partial)
                y.append (y_partial)
        else:
            import math
            roundNum = math.ceil(len(dfX)/windowNum)
            for i in range(roundNum): #This ensures all rows are captured
                dfX_partial = dfX[i*windowNum:(i+1)*windowNum]
                dfy_partial = dfy[i:(i+1)]
                X_partial = dfX_partial.values
                y_partial = dfy_partial.values[0][0]
                X.append (X_partial)
                y.append (y_partial)

        X = np.array(X)
        y = np.array(y)
    
    return X, y

## 2-3. Training Data Transformation

In [12]:
# for regression
if mode_selection == 'regression':
    transformParameter = {}

    # from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf

    trainX, trainy = transDFtoNP_yk(train_x, train_y, windowNum=144)
    valX, valy = transDFtoNP_yk(val_x, val_y, windowNum=144)

# for forecast
elif mode_selection == 'forecast':
    transformParameter = {
    'future_step': 2,
    'past_step': 24,
    'feature_col': featureListX,
    'target_col': featureListy,
    'clean_param': cleanTrainDataParam
    }
    from Clust.clust.transformation.purpose.machineLearning import LSTMData

    LSTMD = LSTMData()
    trainX, trainy = LSTMD.transform_Xy_arr(train_x, transformParameter, transformParameter['clean_param'])
    valX, valy = LSTMD.transform_Xy_arr(val_x, transformParameter, transformParameter['clean_param'])

In [13]:
print(type(trainX), trainX.shape)
print(trainX)

<class 'numpy.ndarray'> (76, 144, 24)
[[[0.12778644 0.32472217 0.25533572 ... 0.10859729 0.609375   0.53846154]
  [0.13494175 0.32862156 0.2581399  ... 0.11010558 0.609375   0.52564103]
  [0.13861114 0.3361279  0.26328088 ... 0.11161388 0.609375   0.51282051]
  ...
  [0.18264379 0.37248976 0.40037389 ... 0.1040724  0.609375   0.38461538]
  [0.18374461 0.37248976 0.34070728 ... 0.11010558 0.609375   0.38461538]
  [0.18007522 0.36859037 0.30954977 ... 0.11613876 0.609375   0.38461538]]

 [[0.33263003 0.49590563 0.40402377 ... 0.34841629 0.609375   0.30769231]
  [0.33740024 0.52364705 0.42218414 ... 0.34992459 0.609375   0.28205128]
  [0.37949336 0.53450965 0.43433557 ... 0.35143288 0.609375   0.25641026]
  ...
  [0.32311583 0.51687881 0.47279282 ... 0.37330317 0.609375   0.07692308]
  [0.3238235  0.51984514 0.47826764 ... 0.36199095 0.609375   0.07692308]
  [0.32641828 0.52198284 0.46678388 ... 0.35067873 0.609375   0.07692308]]

 [[0.49720209 0.6941899  0.74123695 ... 0.3800905  0.60937

In [14]:
print(type(trainy), trainy.shape)
print(trainy)

<class 'numpy.ndarray'> (76,)
[0.67048704 0.35052531 0.         0.35243553 0.37774592 1.
 0.37058262 0.41642789 0.26647563 0.24450811 0.73543454 0.76265515
 0.77125116 0.23447945 0.28939825 0.26074498 0.3008596  0.86341927
 0.28127981 0.39255015 0.45272203 0.53438397 0.69723018 0.35530083
 0.29369625 0.20487105 0.29560647 0.3643744  0.84383952 0.30993312
 0.38347658 0.32951287 0.45176694 0.65281756 0.36485192 0.32139443
 0.25167141 0.57115562 0.156638   0.2034384  0.23495702 0.21394458
 0.44269337 0.00191022 0.44317094 0.68194844 0.66284617 0.5210124
 0.5195797  0.18290353 0.50382044 0.26170007 0.61318051 0.31948421
 0.29608403 0.37583571 0.24689589 0.08070678 0.30038203 0.52865327
 0.78080229 0.79130847 0.78319003 0.55682901 0.80038203 0.35052531
 0.24689589 0.46084047 0.2373448  0.34383952 0.51146126 0.63132755
 0.54584527 0.28557781 0.49379178 0.51002865]


## 2-4 Set Model Parameters & Train Parameters

In [15]:
seq_len, input_size = trainX.shape[1], trainX.shape[2]
print(seq_len, input_size)

# RNN models (RNN, LSTM, GRU) parameters
if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }

trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}

144 24


In [16]:
modelTags =["aaaaa"]
trainDataType = "timeseries"
modelPurpose = "regression"

# # 2
trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# 3. 모델을 저장할 파일 패스를 생성한다.

model_name = 'Test' + model_method
from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
trainDataPathList = [model_name, dataName_X, trainParameter_encode]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

./Models/CNN_1D_rg/TestCNN_1D_rg/IntegraionTrainX/d531eec11664669cff1f6a3ad9639012/model.pkl


## 2-5 Training 

In [17]:
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML

rml = RML()
rml.set_param(trainParameter)
rml.set_model(model_method, modelParameter)
rml.set_data(trainX, trainy, valX, valy)
rml.train()
rml.save_best_model(modelFilePath)

Start training model
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[1/10] Training loss: 0.2118	 Validation loss: 0.1290
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[2/10] Training loss: 0.1689	 Validation loss: 0.0929
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[3/10] Training loss: 0.1228	 Validation loss: 0.0646
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[4/10] Training loss: 0.0767	 Validation loss: 0.0450
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[5/10] Training loss: 0

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[8/10] Training loss: 0.0505	 Validation loss: 0.0498
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[9/10] Training loss: 0.0481	 Validation loss: 0.0529
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
torch.Size([16, 1])
torch.Size([16])
[10/10] Training loss: 0.0525	 Validation loss: 0.0489

Training complete in 0m 0s


## 2-6 Save MetaData 

In [18]:
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
from Clust.setting import influx_setting_KETI as ins
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

# from Clust.clust.transformation.general.dataScaler import encode_hash_style
# trainParameter_encode =  encode_hash_style(str(trainParameter))
if mode_selection == 'regression':
    modelInfoMeta ={
        "trainDataInfo":trainDataInfo,
        "modelName":model_name,
        "featureList":featureListX,
        "target": featureListy,
        "trainDataType":trainDataType,
        "modelPurpose":modelPurpose,
        "model_method":model_method,
        "modelTags":modelTags,
        "cleanTrainDataParam":cleanTrainDataParam,
        "NaNProcessingParam":NaNProcessingParam,
        "trainDataName":[dataName_X, dataName_y],
        "trainParameter": trainParameter,
        "modelParameter": modelParameter,
        "transformParameter":transformParameter,
        "NaNProcessingParam":NaNProcessingParam,
        "scalerParam":scalerParam,
        "files":{
            "modelFile":{
                "fileName":"model.pth",
                "filePath":modelFilePath
            },
            "XScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":X_scalerFilePath       
            },
            "yScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":y_scalerFilePath       
            }
        }
    }

elif mode_selection == 'forecast':
    modelInfoMeta ={
        "trainDataInfo":trainDataInfo,
        "modelName":model_name,
        "featureList":featureListX,
        "target": featureListy,
        "trainDataType":trainDataType,
        "modelPurpose":modelPurpose,
        "model_method":model_method,
        "modelTags":modelTags,
        "cleanTrainDataParam":cleanTrainDataParam,
        "NaNProcessingParam":NaNProcessingParam,
        "trainDataName":[dataName_X],
        "trainParameter": trainParameter,
        "modelParameter": modelParameter,
        "transformParameter":transformParameter,
        "NaNProcessingParam":NaNProcessingParam,
        "scalerParam":scalerParam,
        "files":{
            "modelFile":{
                "fileName":"model.pth",
                "filePath":modelFilePath
            },
            "XScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":X_scalerFilePath       
            }
        }
    }

modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)


========== Data Save Success ==========
{'trainDataInfo': {'startTime': '2016-01-11', 'endTime': '2016-04-15', 'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']], 'processParam': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False, 'param': {}}}, 'imputation_param': {'flag': False, 'imputation_method': [], 'totalNonNanRatio': 80}}, 'integration_freq_sec': 600, 'cleanParam': 'NoClean', 'DataSaveMode': 'influx'}, 'modelName': 'TestCNN_1D_rg', 'featureList': ['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Press_mm_hg', 'RH_out', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed'], 'target': ['value'], 'trainDataType': 'times